In [1]:
# import requests
# from bs4 import BeautifulSoup
# import os
# from concurrent.futures import ThreadPoolExecutor

# url = "https://www.ncei.noaa.gov/data/oceans/argo/gadr/data/indian/2019/01/"
# folder = "./data"
# os.makedirs(folder, exist_ok=True)

# # Get list of .nc files
# r = requests.get(url)
# from bs4 import BeautifulSoup
# soup = BeautifulSoup(r.text, "html.parser")
# files = [node.get("href") for node in soup.find_all("a") if node.get("href").endswith(".nc")]

# def download_file(f):
#     r_file = requests.get(url + f)
#     with open(os.path.join(folder, f), "wb") as file:
#         file.write(r_file.content)
#     print("Downloaded:", f)

# # Download in parallel using 5 threads
# with ThreadPoolExecutor(max_workers=5) as executor:
#     executor.map(download_file, files)

# print("Done")


In [8]:
import xarray as xr

file = r"C:\Users\shubh\Downloads\Classroom\Hackathons\SIH 2025\ARGO\Test\data\nodc_D1900975_339.nc"   # replace with one file you downloaded
ds = xr.open_dataset(file)

ds


<xarray.Dataset> Size: 7kB
Dimensions:                       (n_prof: 1, n_param: 3, n_levels: 72,
                                   n_calib: 1, n_history: 6)
Dimensions without coordinates: n_prof, n_param, n_levels, n_calib, n_history
Data variables: (12/65)
    data_type                     object 8B ...
    format_version                object 8B ...
    handbook_version              object 8B ...
    reference_date_time           object 8B ...
    date_creation                 object 8B ...
    date_update                   object 8B ...
    ...                            ...
    history_parameter             (n_history, n_prof) object 48B ...
    history_start_pres            (n_history, n_prof) float32 24B ...
    history_stop_pres             (n_history, n_prof) float32 24B ...
    history_previous_value        (n_history, n_prof) float32 24B ...
    history_qctest                (n_history, n_prof) object 48B ...
    crs                           int32 4B ...
Attributes: (12/50)
    title:                           Argo float vertical profile
    institution:                     AOML
    source:                          Argo float
    history:                         2019-10-20T04:35:26Z boyer convAGDAC.f90...
    references:                      https://www.nodc.noaa.gov/argo/
    comment:                         
    ...                              ...
    time_coverage_end:               2019-01-06T19:10:56Z
    time_coverage_duration:          point
    time_coverage_resolution:        point
    gadr_ConventionVersion:          GADR-3.0
    gadr_program:                    convAGDAC.f90
    gadr_programVersion:             1.2

In [3]:
# import pandas as pd
# import netCDF4
# import os

# data_list = []

# for file_name in os.listdir("./data"):
#     if file_name.endswith(".nc"):
#         ds = netCDF4.Dataset(os.path.join("./data", file_name))
#         # Extract variables
#         float_id = getattr(ds, 'PLATFORM_NUMBER', 'Unknown')
#         cycle = getattr(ds, 'CYCLE_NUMBER', 'Unknown')
#         lat = ds.variables['LATITUDE'][:]
#         lon = ds.variables['LONGITUDE'][:]
#         pres = ds.variables['PRES'][:]
#         temp = ds.variables['TEMP'][:]
#         psal = ds.variables['PSAL'][:]
        
#         # Flatten and store
#         for i in range(len(lat)):
#             data_list.append({
#                 "float_id": float_id,
#                 "cycle": cycle,
#                 "latitude": lat[i],
#                 "longitude": lon[i],
#                 "pressure": pres[i],
#                 "temperature": temp[i],
#                 "salinity": psal[i]
#             })

# df = pd.DataFrame(data_list)
# df.to_parquet("argo_indian_201901.parquet", index=False)


In [9]:
import xarray as xr

ds = xr.open_dataset(r"C:\Users\shubh\Downloads\Classroom\Hackathons\SIH 2025\ARGO\Test\data\nodc_D1900975_340.nc")

print(ds.variables.keys())    # all variables
print(ds.attrs.keys())        # dataset attributes


KeysView(Frozen({'data_type': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Data type
    conventions:  Argo reference table 1, 'format_version': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:  File format version, 'handbook_version': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:  Data handbook version, 'reference_date_time': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Date of reference for Julian days
    conventions:  YYYYMMDDHHMISS, 'date_creation': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Date of file creation
    conventions:  YYYYMMDDHHMISS, 'date_update': <xarray.Variable ()> Size: 8B
[1 values with dtype=object]
Attributes:
    long_name:    Date of update of this file
    conventions:  YYYYMMDDHHMISS, 'platform_number': <xarray.Variable (n_prof: 1)> Size: 8B
[1 valu

In [4]:
import xarray as xr
import pandas as pd
import glob

files = glob.glob("./data/*.nc")
all_data = []

for f in files:
    ds = xr.open_dataset(f)
    
    float_id = ds['platform_number'].values[0]
    cycle = int(ds['cycle_number'].values[0])
    lat = float(ds['latitude'].values[0])
    lon = float(ds['longitude'].values[0])
    timestamp = pd.to_datetime(ds['juld'].values[0])
    
    pres = ds['pres'].values[0]          # shape: (n_levels,)
    temp = ds['temp'].values[0]
    psal = ds['psal'].values[0]
    
    # Optional QC flags
    pres_qc = ds['pres_qc'].values[0]
    temp_qc = ds['temp_qc'].values[0]
    psal_qc = ds['psal_qc'].values[0]
    
    n_levels = pres.shape[0]
    
    for i in range(n_levels):
        all_data.append({
            "float_id": float_id,
            "cycle": cycle,
            "latitude": lat,
            "longitude": lon,
            "timestamp": timestamp,
            "pressure": pres[i],
            "temperature": temp[i],
            "salinity": psal[i],
            "pressure_qc": pres_qc[i],
            "temperature_qc": temp_qc[i],
            "salinity_qc": psal_qc[i]
        })

df = pd.DataFrame(all_data)
df.to_parquet("argo_indian_201901.parquet", index=False)


KeyError: "No variable named 'platform_number'. Variables on the dataset include ['DATA_TYPE', 'FORMAT_VERSION', 'HANDBOOK_VERSION', 'REFERENCE_DATE_TIME', 'DATE_CREATION', ..., 'HISTORY_PARAMETER', 'HISTORY_START_PRES', 'HISTORY_STOP_PRES', 'HISTORY_PREVIOUS_VALUE', 'HISTORY_QCTEST']"